In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import random

from sklearn.model_selection import train_test_split

# scikit-learn k-fold cross-validation
from numpy import array
from sklearn.model_selection import KFold

from sklearn import metrics

## Load input data

In [2]:
#Hardcoded values
timestep = 16
k=10

In [3]:
path = 'MovementAAL/dataset/MovementAAL_RSS_'

sequences = list()
file_targets = pd.read_csv('MovementAAL/dataset/MovementAAL_target.csv')

targets = list()

for i in range(1,315):
    file_path = path + str(i) + '.csv'
    file = pd.read_csv(file_path, header=0)
    for j in range(0, (len(file) - timestep)+1):
        arr = file[j:j+timestep]
        sequences.append(arr)
        targets.append(file_targets.values[:,1][i-1])

In [4]:
print(len(sequences))
print(len(targets))
sequences[0]

8487
8487


,RSS_anchor1,RSS_anchor2,RSS_anchor3,RSS_anchor4
0,-0.904760,-0.48,0.285710,0.30
1,-0.571430,-0.32,0.142860,0.30
2,-0.380950,-0.28,-0.142860,0.35
3,-0.285710,-0.20,-0.476190,0.35
4,-0.142860,-0.20,0.142860,-0.20
5,-0.142860,-0.20,0.047619,0.00
6,-0.142860,-0.16,-0.380950,0.20
7,-0.142860,-0.04,-0.619050,-0.20
8,-0.095238,-0.08,0.142860,-0.55
9,-0.047619,0.04,-0.095238,0.05


In [5]:
groups = pd.read_csv('MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]

In [6]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()


count    8487.0
mean       16.0
std         0.0
min        16.0
25%        16.0
50%        16.0
75%        16.0
max        16.0
dtype: float64

In [7]:
accuracy_tab = list()

#k-folder cross validation
kfold = KFold(n_splits=k)
kfold.get_n_splits(sequences)

print(kfold)

KFold(n_splits=10, random_state=None, shuffle=False)


In [8]:
sequences = np.array(sequences)
targets = np.array(targets)
num = 1

for train_index, test_index in kfold.split(sequences):
    
    X_train, X_test = sequences[train_index], sequences[test_index]
    Y_train, Y_test = targets[train_index], targets[test_index]

    
    X_train = np.array(X_train)
    X_test = np.array(X_test)

    Y_train = np.array(Y_train)
    Y_train = (Y_train+1)/2

    Y_test = np.array(Y_test)
    Y_test = (Y_test+1)/2
    
    model = Sequential()
    model.add(LSTM(256, input_shape=(timestep, 4),dropout=0.1))
    model.add(Dense(1, activation='sigmoid'))

    adam = Adam(learning_rate=0.001)
    chk = ModelCheckpoint('best_model'+ str(num) +'.pkl', monitor='accuracy', save_best_only=True, mode='max', verbose=1)

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.fit(X_train, Y_train, epochs=20, batch_size=512, callbacks=[chk])
    
    model = load_model('best_model'+str(num)+'.pkl')
    test_preds = (model.predict(X_test) > 0.5).astype("int32")

    accuracy_tab.append(metrics.accuracy_score(Y_test, test_preds))
    num=num+1

Epoch 1/20
30/30 [==============================] - ETA: 0s - loss: 0.6052 - accuracy: 0.6867
Epoch 1: accuracy improved from -inf to 0.68670, saving model to best_model1.pkl


INFO:tensorflow:Assets written to: best_model1.pkl\assets


INFO:tensorflow:Assets written to: best_model1.pkl\assets


30/30 [==============================] - 12s 308ms/step - loss: 0.6052 - accuracy: 0.6867
Epoch 2/20
30/30 [==============================] - ETA: 0s - loss: 0.5117 - accuracy: 0.7533
Epoch 2: accuracy improved from 0.68670 to 0.75334, saving model to best_model1.pkl


INFO:tensorflow:Assets written to: best_model1.pkl\assets


...

INFO:tensorflow:Assets written to: best_model10.pkl\assets


30/30 [==============================] - 9s 296ms/step - loss: 0.2677 - accuracy: 0.8817
Epoch 18/20
30/30 [==============================] - ETA: 0s - loss: 0.2649 - accuracy: 0.8771
Epoch 18: accuracy did not improve from 0.88166
30/30 [==============================] - 4s 143ms/step - loss: 0.2649 - accuracy: 0.8771
Epoch 19/20
30/30 [==============================] - ETA: 0s - loss: 0.2591 - accuracy: 0.8813
Epoch 19: accuracy did not improve from 0.88166
30/30 [==============================] - 4s 146ms/step - loss: 0.2591 - accuracy: 0.8813
Epoch 20/20
30/30 [==============================] - ETA: 0s - loss: 0.2383 - accuracy: 0.8878
Epoch 20: accuracy improved from 0.88166 to 0.88781, saving model to best_model10.pkl


INFO:tensorflow:Assets written to: best_model10.pkl\assets


INFO:tensorflow:Assets written to: best_model10.pkl\assets


27/27 [==============================] - 1s 22ms/step


In [9]:
print(accuracy_tab)

[0.7656065959952886, 0.7408716136631331, 0.6148409893992933, 0.773851590106007, 0.8857479387514723, 0.8256772673733804, 0.6383981154299175, 0.2865566037735849, 0.9775943396226415, 0.9257075471698113]


In [10]:
model = load_model('best_model.pkl')
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

test_preds = (model.predict(X_test) > 0.5).astype("int32")

classification_report = metrics.classification_report(Y_test, test_preds)
print(classification_report)

OSError: No file or directory found at best_model.pkl